## 1. 데이터 로드와 전처리

In [1]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
vocab_size = 20000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)
print('훈련용 리뷰 개수 :',len(X_train))
print('테스트용 리뷰 개수 :',len(X_test))

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
훈련용 리뷰 개수 : 25000
테스트용 리뷰 개수 : 25000


In [3]:
print('훈련 데이터의 첫번째 샘플 :',X_train[0])
print('훈련 데이터의 첫번째 샘플의 레이블 :',y_train[0])

훈련 데이터의 첫번째 샘플 : [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
훈련 데이터의 첫번째 샘플의 레이블 : 1


In [5]:
print('훈련용 리뷰의 평규 길이: {}'.format(np.mean(list(map(len, X_train)), dtype=int)))
print('테스트용 리뷰의 평균 길이: {}'.format(np.mean(list(map(len, X_test)), dtype=int)))

훈련용 리뷰의 평규 길이: 238
테스트용 리뷰의 평균 길이: 230


In [6]:
max_len = 400

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
print('X_train의 크기(shape) :', X_train.shape)
print('X_test의 크기(shape) :', X_test.shape)

X_train의 크기(shape) : (25000, 400)
X_test의 크기(shape) : (25000, 400)


## 2. 모델 설계하기

In [7]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 64

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))

# 모든 단어 벡터의 평균을 구한다.
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('embedding_average_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, batch_size=32, epochs=10, callbacks=[es, mc], validation_split=0.2)

Epoch 1/10
623/625 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - acc: 0.5763 - loss: 0.6727
Epoch 1: val_acc improved from -inf to 0.76680, saving model to embedding_average_model.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - acc: 0.5766 - loss: 0.6725 - val_acc: 0.7668 - val_loss: 0.5214
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - acc: 0.8125 - loss: 0.4654
Epoch 2: val_acc improved from 0.76680 to 0.81880, saving model to embedding_average_model.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - acc: 0.8125 - loss: 0.4654 - val_acc: 0.8188 - val_loss: 0.3975
Epoch 3/10
623/625 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - acc: 0.8684 - loss: 0.3376
Epoch 3: val_acc improved from 0.81880 to 0.87360, saving model to embedding_average_model.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - acc: 0.8684 - loss: 0.3375 - val_acc: 0.8736 - val_loss: 0.3232
Epoch 4/10
624/625 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.8938 - loss: 0.2773
Epoch 4: val_acc improved from 0.87360 to 0.88200, saving model to embedding_average_model.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - acc: 0.8938 - loss: 0.2772 - val_acc: 0.8820 - val_loss: 0.3002
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - acc: 0.9071 - loss: 0.2435
Epoch 5: val_acc did not improve from 0.88200
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - acc: 0.9071 - loss: 0.2435 - val_acc: 0.8804 - val_loss: 0.2956
Epoch 6/10
622/625 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - acc: 0.9221 - loss: 0.2092
Epoch 6: val_acc did not improve from 0.88200
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - acc: 0.9221 - loss: 0.2093 - val_acc: 0.8596 - val_loss: 0.3221
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - acc: 0.9206 - loss: 0.2049
Epoch 7: val_acc improved from 0.88200 to 0.89260, saving model to embedding_average_model.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - acc: 0.9206 - loss: 0.2048 - val_acc: 0.8926 - val_loss: 0.2780
Epoch 8/10
623/625 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - acc: 0.9380 - loss: 0.1711
Epoch 8: val_acc improved from 0.89260 to 0.89360, saving model to embedding_average_model.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - acc: 0.9380 - loss: 0.1711 - val_acc: 0.8936 - val_loss: 0.2764
Epoch 9/10
622/625 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - acc: 0.9429 - loss: 0.1532
Epoch 9: val_acc did not improve from 0.89360
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - acc: 0.9429 - loss: 0.1532 - val_acc: 0.8834 - val_loss: 0.2965
Epoch 10/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - acc: 0.9480 - loss: 0.1410
Epoch 10: val_acc improved from 0.89360 to 0.89500, saving model to embedding_average_model.h5


625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - acc: 0.9480 - loss: 0.1410 - val_acc: 0.8950 - val_loss: 0.2811


In [8]:
loaded_model = load_model('embedding_average_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - acc: 0.8863 - loss: 0.2925

 테스트 정확도: 0.8860
